In [15]:
!pip install grad-cam


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
cam_port = '/content/drive/MyDrive/HackIA_2024_GRP_2/Dataset/No_fire_analysis.mp4'

In [24]:
filename = '/content/drive/MyDrive/HackIA_2024_GRP_2/Models/full_model_MobileNet_V3_Small_Weights_test_acc_0.923.pth'

In [25]:
import cv2
import torch
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import numpy as np
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

# Open Model and GradCam
model = torch.load(filename)
model.eval()

for param in model.parameters():
        param.requires_grad = True

# Définir la couche cible pour Grad-CAM
target_layers = [model.features[-1]]

cam = GradCAM(model=model, target_layers=target_layers)

# Class to use (cats)
targets = [ClassifierOutputTarget(0)]

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(cam_port)


# Check if camera opened successfully
if (cap.isOpened() == False):
    print("Error opening video stream or file")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (400,400))

# Read until video is completed
while (cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:

        rgb_img = cv2.resize(frame, (224, 224))
        rgb_img = np.float32(rgb_img) / 255
        input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

        # In this example grayscale_cam has only one image in the batch:
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        visualization = cv2.resize(visualization, (400, 400))

        # # Display the resulting frame
        # cv2.imshow('Frame', visualization)

        # # Press Q on keyboard to  exit
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

        out.write(visualization)

    # Break the loop
    else:
        break